# Pycorrelate examples and validation

This notebook show `pycorrelate` usage as well comparisons with other 
less efficient implementations.

In [ ]:
import numpy as np
import h5py

In [ ]:
# Tweak here matplotlib style
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.sans-serif'].insert(0, 'Arial')
mpl.rcParams['font.size'] = 14
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [ ]:
import pycorrelate as pyc

# Load Data

Download data [here](http://files.figshare.com/2182601/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5).

In [ ]:
fname = "./data/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5"
h5 = h5py.File(fname)
unit = 12.5e-9

In [ ]:
num_ph = int(10e6)
t = h5['photon_data']['timestamps'][:num_ph][h5['photon_data']['detectors'][:num_ph] == 0]
u = h5['photon_data']['timestamps'][:num_ph][h5['photon_data']['detectors'][:num_ph] == 1]
# In the algorithm I assume t[0] < u[0], if not true swap u and v
if t[0] > u[0]:    
    t, u = u, t

In [ ]:
t.shape, u.shape, t[0], u[0]

In [ ]:
t.max()*unit, u.max()*unit

Timestamps need to be monotonic, let's test it:

In [ ]:
assert (np.diff(t) > 0).all()
assert (np.diff(u) > 0).all()

# Log-scale bins (base 10)

In [ ]:
exp10_min = -7
exp10_max = 0
points_per_base = 10

num_points = points_per_base*(exp10_max - exp10_min) + 1
bins = np.logspace(exp10_min, exp10_max, num_points) / unit
nbins = len(bins) -1

In [ ]:
nbins, bins[:4], bins[-4:], bins[-1]*unit

In [ ]:
G = pyc.correlate(t, u, bins)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(bins*unit, np.hstack((G[:1], G)), drawstyle='steps-pre')
plt.xlabel('Time (s)')
#for x in bins[1:]: plt.axvline(x*unit, lw=0.2)  # to mark bins
plt.xlim(30e-9, 2)
plt.xscale('log')
plt.grid(True)
plt.grid(True, which='minor', lw=0.3)

# Log-scale bins (base 2)

In [ ]:
exp2_min = -1
exp2_max = 28
points_per_base = 4

num_points = points_per_base*(exp2_max - exp2_min) + 1
bins = np.unique(
    np.logspace(exp2_min, exp2_max, num_points, 
                base=2, dtype='int64'))
nbins = len(bins) -1

In [ ]:
nbins, bins[:4], bins[-4:], bins[-1]*unit

In [ ]:
G = pyc.correlate(t, u, bins)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(bins*unit, np.hstack((G[:1], G)), drawstyle='steps-pre')
plt.xscale('log')
plt.xlabel('Time (s)')
#for x in bins[1:]: plt.axvline(x*unit, lw=0.2)  # to mark bins
plt.xlim(30e-9, 2)
plt.grid(True)
plt.grid(True, which='minor', lw=0.3)

# Multi-tau bins

In [ ]:
n_group = 4
bin_widths = []
for i in range(26):
    bin_widths += [2**i]*n_group
np.array(bin_widths)
bins = np.hstack(([0], np.cumsum(bin_widths)))
nbins = len(bins) - 1

In [ ]:
nbins, bins[:4], bins[-4:], bins[-1]*unit

In [ ]:
G = pyc.correlate(t, u, bins)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(bins*unit, np.hstack((G[:1], G)), drawstyle='steps-pre')
plt.xscale('log')
plt.xlabel('Time (s)')
#for x in bins[1:]: plt.axvline(x*unit, lw=0.2)  # to mark bins
plt.xlim(30e-9, 2)
plt.grid(True)
plt.grid(True, which='minor', lw=0.3)

# Test: comparison with np.histogram

For testing we use smaller input arrays:

In [ ]:
tt = t[:10000]
uu = u[:10000]

The same algorims can be expressed in numpy in an incredible simple way
using `np.histogram`:

In [ ]:
Y = np.zeros(nbins, dtype=np.int64)
for ti in tt:
    Yc, _ = np.histogram(uu - ti, bins=bins)
    Y += Yc
G = Y / np.diff(bins)

In [ ]:
assert (G == pyc.correlate(tt, uu, bins)).all()

Test passed! Here we demonstrated that the logic of the algorithm
is implemented as described in the paper (and in the few lines of code above).

# Tests: comparison with np.correlate

The comparison with `np.correlate` is a little tricky.
First we need to bin our input to create timetraces that can be correlated
by linear convolution. For testing purposes, let's choice
some timetrace bins:

In [ ]:
bins_tt = np.arange(0, tt.max()*unit, 50e-6) / unit
bins_uu = np.arange(0, uu.max()*unit, 50e-6) / unit

In [ ]:
bins_tt.max()*unit, bins_tt.size

In [ ]:
bins_uu.max()*unit, bins_uu.size

In [ ]:
tx, _ = np.histogram(tt, bins=bins_tt)
ux, _ = np.histogram(uu, bins=bins_uu)

plt.plot(bins_tt[1:]*unit, tx)
plt.plot(bins_uu[1:]*unit, ux)
plt.xlabel('Time (s)')

The plots above are the two curves we are going to feed to
`np.correlate`:

In [ ]:
C = np.correlate(tx, ux, mode='full')

We need to trim the result to obtain a proper alignment with
the 0-time lag:

In [ ]:
CC = C[ux.size-1:]  # trim to positive time lags

Now, let's compute the correlation with pycorrelate using the same
bin-width used for the timetrace:

In [ ]:
bins_g = np.arange(0, 0.5, 50e-6) / unit
G = pyc.correlate(tt, uu, bins_g)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(np.arange(1, CC.size+1)*50e-3, CC/4000, lw=2, label='numpy.correlate') 
plt.plot(bins_g[1:]*unit*1e3, G, alpha=0.7, lw=2, label='pycorrelate.correlate')
plt.xlabel('Time (ms)', fontsize='large')
plt.grid(True)
plt.xlim(30e-3, 500)
plt.xscale('log')
plt.title('pycorrelate.correlate vs numpy.correlate', fontsize='x-large')
plt.legend(loc='best', fontsize='x-large')

Given the widely different nature of the two algorithms, 
this match is a fairly good evidence of correctness.